# This is a Modeller Batch Script
## Script made under guidance of Rawal Genomics Team
### Script made by Prateek Paul
Source: https://salilab.org/modeller/tutorial/basic.html

#### #pip install Bio
##### Install Modeller from https://salilab.org/modeller/download_installation.html
##### Also download suplementary files and put them in same folder https://salilab.org/modeller/supplemental.html
##### licence key https://salilab.org/modeller/registration.html
#### code references https://salilab.org/modeller/tutorial/basic.html

In [19]:
#importing all modelues
import os
import glob
import Bio
import urllib
import shutil, sys 
import numpy as np
import pandas as pd
from Bio.PDB import *
from Bio import SeqIO
from modeller import *
from modeller.automodel import *
#sys.stdout = open("dump.txt", "w")

In [20]:
#creating folder so that main folder looks clean
def folder():
    try:
        os.mkdir('Ready')
        os.mkdir('ali')
        os.mkdir('buildprofile')
        os.mkdir('PDB')
    except:
        print('folder exists')

In [21]:
#generating main sequence ali files
def aligen(ID, Seq):
    ids,v=ID.split('.')
    file1 = open("ali/"+ids+".ali", "w")
    file1.write(">P1;"+ids+"\n")
    file1.write("sequence:"+ids+":::::::0.00: 0.00"+"\n")
    file1.write(Seq+"*")
    file1.close()

In [22]:
#Searching for structures related to the main sequence 
def build_profile(ID):
    ids,v=ID.split('.')
    log.verbose()
    env = Environ()
    sdb = SequenceDB(env)
    sdb.read(seq_database_file='pdb_95.pir', seq_database_format='PIR',
             chains_list='ALL', minmax_db_seq_len=(30, 4000), clean_sequences=True)
    sdb.write(seq_database_file='pdb_95.bin', seq_database_format='BINARY',
              chains_list='ALL')
    sdb.read(seq_database_file='pdb_95.bin', seq_database_format='BINARY',
             chains_list='ALL')
    aln = Alignment(env)
    aln.append(file="ali/"+ids+".ali", alignment_format='PIR', align_codes='ALL')
    prf = aln.to_profile()
    prf.build(sdb, matrix_offset=-450, rr_file='${LIB}/blosum62.sim.mat',
              gap_penalties_1d=(-500, -50), n_prof_iterations=1,
              check_profile=False, max_aln_evalue=0.01)
    prf.write(file="buildprofile/"+ids+'build_profile.prf', profile_format='TEXT')
    aln = prf.to_alignment()
    aln.write(file="buildprofile/"+ids+'build_profile.ali', alignment_format='PIR')

In [23]:
#Downloading the selected pdb files required for the templetes
def pdbDn(pdbid):
    urllib.request.urlretrieve('http://files.rcsb.org/download/'+pdbid+'.pdb', pdbid+'.pdb')

In [24]:
#comparing the downloaded pdb
#presently this step is being skipped 
#and the code is taking the top hit for the pdb 
def compare(listpdb,ID):
    ids,v=ID.split('.')
    env = Environ()
    aln = Alignment(env)
    for (pdb, chain) in (listpdb):
        m = Model(env, file=pdb, model_segment=('FIRST:'+chain, 'LAST:'+chain))
        aln.append_model(m, atom_files=pdb, align_codes=pdb+chain)
    aln.malign()
    aln.malign3d()
    aln.compare_structures()
    aln.id_table(matrix_file='family.mat')
    env.dendrogram(matrix_file='family.mat', cluster_cut=-1.0)

In [25]:
#Aligning Main seq with the template
def align2d(pdbid,chain,ID):
    ids,v=ID.split('.')
    env = Environ()
    aln = Alignment(env)
    mdl = Model(env, file=pdbid, model_segment=('FIRST:A','LAST:A'))
    aln.append_model(mdl, align_codes=pdbid+chain, atom_files=pdbid+'.pdb')
    aln.append(file="ali/"+ids+'.ali', align_codes=ids)
    aln.align2d()
    aln.write(file="ali/"+ids+'-'+pdbid+chain+'.ali', alignment_format='PIR')
    aln.write(file="ali/"+ids+'-'+pdbid+chain+'.pap', alignment_format='PAP')

In [26]:
#building the profile for the selected pdb to main seq
def prf(ids):
        a_file = open("buildprofile/"+ids+'build_profile.prf', "r")
        lines = a_file.readlines()
        a_file.close()
        line=lines[6:]
        f = open("buildprofile/"+ids+'build_profile.prf', "w")
        #f.write('sno \t id \t l1 \t l2')
        for x in line:
            for z in x.split():
                f.write(z+'\t')
            f.write('\n')
        f.close()
        df = pd.read_csv("buildprofile/"+ids+'build_profile.prf', sep='\t', header=None)
        #return (len(df))
        df=df.sort_values(by=([10,11]), ascending=(False,False))
        df=df.reset_index()
        identity = df[10]
        #y=identity[1]
        if len(df) > 0: 
            pdbids=df[1]
            if len(df) > 1:
                x=pdbids[1]
                pdbDn(x[:4])
            else:
                x = 'cannot'
        else:
            x = 'cannot'        
        return (len(df)), identity,x

In [27]:
#Model building
def model(ali,ID,pdbid):
    ids,v=ID.split('.')
    env = Environ()
    a = AutoModel(env, alnfile=ali,
                  knowns=pdbid, sequence=ids,
                  assess_methods=(assess.DOPE,
                                  assess.GA341))
    a.starting_model = 1
    a.ending_model = 5
    a.make()

In [28]:
#Model evaluation and removing of the other files
def eva(ids):
    dope = 0.0
    for file in glob.glob(ids+".*.pdb"):
        fh = open(file)
        for each in fh:
            if "REMARK   6 DOPE score:" in each:
                b = each.split(' ')
                r = b[6].strip()
                if float(r) < float(dope):
                    dope=b[6]
                    mainfile=file
        fh.close()
    pdbs=glob.glob(ids+".*.pdb")
    shutil.move(mainfile, 'Ready')
    for pdbmod in glob.glob(ids+".*"):
        os.remove(pdbmod)

In [29]:
def main():
    folder()
    
    with open("AAHK01.fasta") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            def model(ali,ID,pdbid):
                ids,v=ID.split('.')
                env = Environ()
                a = AutoModel(env, alnfile=ali,
                              knowns=pdbid, sequence=ids,
                              assess_methods=(assess.DOPE,
                                              assess.GA341))
                a.starting_model = 1
                a.ending_model = 5
                a.make()
            
            
            ID=record.id
            Seq=record.seq
            Seq=str(Seq)
            ids,v=ID.split('.')
            aligen(ID, Seq)

            build_profile(ID)
            
            length,identity,x=prf(ids)
            
            if x == 'cannot':
                print('next file')
            else:
                if length > 1:
                    if int(identity[1]) > 34:
                        align2d(x[:4],x[4:5],ID)
                        path = 'ali/'+ids+'-'+x+'.ali'
                        model(path,ID,x)
                        eva(ids)
                        
    #pdbsall=glob.glob("*.pdb")
    for pdbsall in glob.glob("*.pdb"):
        shutil.move(pdbsall, 'PDB')

In [30]:
if __name__ == "__main__":
    main()

folder exists
openf___224_> Open           $(LIB)/restyp.lib
openf___224_> Open           ${MODINSTALL10v0}/modlib/resgrp.lib
rdresgr_266_> Number of residue groups:        2
openf___224_> Open           ${MODINSTALL10v0}/modlib/sstruc.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       191566     187.076     0.183

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       192094     187.592     0.183
openf___224_> Open           ${MODINSTALL10v0}/modlib/resdih.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       240694     235.053     0.230
rdrdih__263_> Number of dihedral angle types         :        9
              Maximal number of dihedral angle optima:        3
              Dihedral angle names                   :  Alph Phi Psi Omeg chi1 chi2 chi3 chi4 chi5
openf___224_> Open           ${MODINSTALL10v0}/modlib/radii.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       253994     248.041     0.242
openf___224_> Op

openf___224_> Open           $(LIB)/restyp.lib
openf___224_> Open           ${MODINSTALL10v0}/modlib/resgrp.lib
rdresgr_266_> Number of residue groups:        2
openf___224_> Open           ${MODINSTALL10v0}/modlib/sstruc.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       191566     187.076     0.183

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       192094     187.592     0.183
openf___224_> Open           ${MODINSTALL10v0}/modlib/resdih.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       240694     235.053     0.230
rdrdih__263_> Number of dihedral angle types         :        9
              Maximal number of dihedral angle optima:        3
              Dihedral angle names                   :  Alph Phi Psi Omeg chi1 chi2 chi3 chi4 chi5
openf___224_> Open           ${MODINSTALL10v0}/modlib/radii.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       253994     248.041     0.242
openf___224_> Open           $


mkilst______> segment topology constructed from sequence and RTF:
              segments residues  atoms  bonds angles dihedrals impropers:
                     1      653   5105   5179      0         0      2058

Dynamically allocated memory at       amaxmodel [B,KiB,MiB]:      2502438    2443.787     2.387

patch_______> segment topology patched using RTF:  1 ;  MET  ;  NTER
              segments residues  atoms  bonds angles dihedrals impropers:
                     1      653   5105   5179   6984      8161      2058


Dynamically allocated memory at       amaxmodel [B,KiB,MiB]:      2731454    2667.436     2.605
patch_______> segment topology patched using RTF:  653 ;  LEU  ;  CTER
              segments residues  atoms  bonds angles dihedrals impropers:
                     1      653   5106   5180   6986      8163      2059

genseg______> segment topology constructed from sequence and RTF:
              segments residues  atoms  bonds angles dihedrals impropers:
               


Dynamically allocated memory at  amaxrestraints [B,KiB,MiB]:      7723059    7542.050     7.365

Dynamically allocated memory at  amaxrestraints [B,KiB,MiB]:      8247347    8054.050     7.865
make_re_422_> Number of previous, current restraints         :    24523    34873
make_re_423_> Number of previous, current selected restraints:    23847    34197
make_re_417_> Restraint type to be calculated:  DISTANCE

Dynamically allocated memory at  amaxrestraints [B,KiB,MiB]:      9295923    9078.050     8.865
make_re_422_> Number of previous, current restraints         :    34873    45174
make_re_423_> Number of previous, current selected restraints:    34197    44498
make_re_417_> Restraint type to be calculated:  DISTANCE

Dynamically allocated memory at  amaxrestraints [B,KiB,MiB]:      9820211    9590.050     9.365
make_re_422_> Number of previous, current restraints         :    45174    50505
make_re_423_> Number of previous, current selected restraints:    44498    49829
make_re_417_


Dynamically allocated memory at amaxgroup_restr [B,KiB,MiB]:     13032299   12726.854    12.429

Dynamically allocated memory at amaxgroup_restr [B,KiB,MiB]:     13083983   12777.327    12.478

Dynamically allocated memory at amaxgroup_restr [B,KiB,MiB]:     13118555   12811.089    12.511

Dynamically allocated memory at amaxgroup_restr [B,KiB,MiB]:     13249371   12938.839    12.636

Dynamically allocated memory at amaxgroup_restr [B,KiB,MiB]:     14578395   14236.714    13.903
read_pa_232_> parameters    BONDS   ANGLS  DIHEDS IMPROPS    MODE
                                0       0       0       0   12561
>> Model assessment by DOPE potential
iatmcls_286W> MODEL atom not classified:  LEU:OXT  LEU

Dynamically allocated memory at  amaxrestraints [B,KiB,MiB]:     14993711   14642.296    14.299

Dynamically allocated memory at  amaxrestraints [B,KiB,MiB]:     15238799   14881.640    14.533

Dynamically allocated memory at  amaxrestraints [B,KiB,MiB]:     15728975   15360.327    15.000

...


DISTANCE1:  0.00 2.10 2.20 2.30 2.40 2.50 2.60 2.70 2.80 2.90 3.00 3.10 3.20 3.30 3.40
DISTANCE2:  2.10 2.20 2.30 2.40 2.50 2.60 2.70 2.80 2.90 3.00 3.10 3.20 3.30 3.40 3.50
FREQUENCY:     0    0    0    0    0   12   44  165  230  376  316  419  484  590  619


<< end of ENERGY.
openf___224_> Open           EAO00202.B99990002.pdb
wrpdb___568_> Residues, atoms, selected atoms:      485     3762     3762
openf___224_> Open           EAO00202.ini

Dynamically allocated memory at       amaxmodel [B,KiB,MiB]:     89437713   87341.516    85.294
read_mo_297_> Segments, residues, atoms:        1      485     3762
read_mo_298_> Segment:        1  1:A  485:A     3762
randomi_498_> Atoms,selected atoms,random_seed,amplitude:     3762     3762        1        4.0000
randomi_496_> Amplitude is > 0; randomization is done.
pick_re_612_> Number of MODEL atoms, selected restraints;     3762    22222
pick_re_612_> Number of MODEL atoms, selected restraints;     3762    24895
pick_re_612_> Number of MO

pick_re_612_> Number of MODEL atoms, selected restraints;     3762    24895
pick_re_612_> Number of MODEL atoms, selected restraints;     3762    26327
pick_re_612_> Number of MODEL atoms, selected restraints;     3762    27906
pick_re_612_> Number of MODEL atoms, selected restraints;     3762    29602
pick_re_612_> Number of MODEL atoms, selected restraints;     3762    31568
pick_re_612_> Number of MODEL atoms, selected restraints;     3762    34587
pick_re_612_> Number of MODEL atoms, selected restraints;     3762    36710
pick_re_612_> Number of MODEL atoms, selected restraints;     3762    38160
pick_re_612_> Number of MODEL atoms, selected restraints;     3762    41001
pick_re_612_> Number of MODEL atoms, selected restraints;     3762    42797
pick_re_612_> Number of MODEL atoms, selected restraints;     3762    44542
iupac_m_487_> NH1/2 swapped:      176.1503        6  6
iupac_m_485_> OD1/2 will be swapped:     -140.1014        7  7
iupac_m_487_> NH1/2 swapped:     -169.1562    


DISTANCE1:  0.00 2.10 2.20 2.30 2.40 2.50 2.60 2.70 2.80 2.90 3.00 3.10 3.20 3.30 3.40
DISTANCE2:  2.10 2.20 2.30 2.40 2.50 2.60 2.70 2.80 2.90 3.00 3.10 3.20 3.30 3.40 3.50
FREQUENCY:     0    0    0    0    0   13   48  138  238  331  332  397  502  613  611


<< end of ENERGY.
openf___224_> Open           EAO00202.B99990004.pdb
wrpdb___568_> Residues, atoms, selected atoms:      485     3762     3762
openf___224_> Open           EAO00202.ini

Dynamically allocated memory at       amaxmodel [B,KiB,MiB]:     89437713   87341.516    85.294
read_mo_297_> Segments, residues, atoms:        1      485     3762
read_mo_298_> Segment:        1  1:A  485:A     3762
randomi_498_> Atoms,selected atoms,random_seed,amplitude:     3762     3762        1        4.0000
randomi_496_> Amplitude is > 0; randomization is done.
pick_re_612_> Number of MODEL atoms, selected restraints;     3762    22222
pick_re_612_> Number of MODEL atoms, selected restraints;     3762    24895
pick_re_612_> Number of MO


>> Summary of successfully produced models:
Filename                          molpdf     DOPE score    GA341 score
----------------------------------------------------------------------
EAO00202.B99990001.pdb        2546.76245   -54215.82422        1.00000
EAO00202.B99990002.pdb        2607.15234   -53857.07031        1.00000
EAO00202.B99990003.pdb        2768.77051   -53809.74609        1.00000
EAO00202.B99990004.pdb        2696.59424   -53850.43359        1.00000
EAO00202.B99990005.pdb        3123.17163   -53401.25000        1.00000

openf___224_> Open           $(LIB)/restyp.lib
openf___224_> Open           ${MODINSTALL10v0}/modlib/resgrp.lib
rdresgr_266_> Number of residue groups:        2
openf___224_> Open           ${MODINSTALL10v0}/modlib/sstruc.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:     89629055   87528.375    85.477

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:     89629583   87528.891    85.477
openf___224_> Open           ${MODINS

HTTPError: HTTP Error 404: Not Found